### mm과 std 방법 중 standardscaler로 데이터 전처리를 한 이유

## 분석 과정 
1. **데이터 읽기 및 분리**
    - 데이터를 읽어들이고 훈련 세트와 테스트 세트로 분리합니다. 
    - 여기서는 80:20의 비율로 분리하였습니다.

2. **MinMaxScaler 사용**
    - MinMaxScaler를 사용하여 훈련 데이터를 [0, 1] 범위로 스케일링합니다.
    - SVM 모델을 훈련하고 성능을 평가합니다.

3. **StandardScaler 사용**
    - StandardScaler를 사용하여 훈련 데이터를 평균 0, 표준편차 1을 갖도록 스케일링합니다.
    - SVM 모델을 훈련하고 성능을 평가합니다.

4. **결과 비교 및 결론 도출**
    - 두 스케일링 방법을 사용하여 얻은 결과를 비교합니다.
    - 이 데이터 세트에 대해 StandardScaler가 더 나은 성능을 보여주었습니다.

### 결론:
MinMaxScaler를 사용한 SVM 모델의 정확도는 약 72.73%이고, StandardScaler를 사용한 SVM 모델의 정확도는 약 81.82%입니다. 따라서 이 데이터 세트에 대해 StandardScaler를 사용하여 데이터를 전처리하는 것이 더 적합하다고 결론짓습니다.

In [3]:
# 필요한 라이브러리 임포트
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 데이터 읽기 및 분리
data = pd.read_csv("../../dataset/clean_feature/model_df.csv")
data = data.drop(columns=['date'])
X = data.drop(columns=['VKOSPI_Label'])
y = data['VKOSPI_Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# MinMaxScaler 사용
mm_scaler = MinMaxScaler()
X_train_mm = mm_scaler.fit_transform(X_train)
X_test_mm = mm_scaler.transform(X_test)
svm_mm = SVC(kernel='linear', random_state=42)
svm_mm.fit(X_train_mm, y_train)
y_pred_mm = svm_mm.predict(X_test_mm)
accuracy_mm = accuracy_score(y_test, y_pred_mm)
print(accuracy_mm)

# StandardScaler 사용
std_scaler = StandardScaler()
X_train_std = std_scaler.fit_transform(X_train)
X_test_std = std_scaler.transform(X_test)
svm_std = SVC(kernel='linear', random_state=42)
svm_std.fit(X_train_std, y_train)
y_pred_std = svm_std.predict(X_test_std)
accuracy_std = accuracy_score(y_test, y_pred_std)

print(accuracy_std)


0.7272727272727273
0.8181818181818182


알겠습니다. SVM 모델의 최적의 파라미터를 찾기 위해 `Pipeline`과 `GridSearchCV`를 활용할 수 있습니다. 

다음은 작업의 개요입니다:

1. **Pipeline 생성**
    - `StandardScaler`와 `SVC`를 포함하는 파이프라인을 생성합니다.

2. **파라미터 그리드 정의**
    - SVM의 주요 파라미터인 `C`와 `kernel`, 그리고 `gamma`에 대한 여러 가지 값을 포함하는 파라미터 그리드를 정의합니다.

3. **GridSearchCV 사용**
    - 파이프라인과 파라미터 그리드를 사용하여 `GridSearchCV`를 설정하고 실행하여 최적의 파라미터를 찾습니다.

4. **최적의 모델 평가**
    - 최적의 파라미터를 사용하는 모델의 성능을 평가합니다.

다음은 해당 작업을 수행하는 코드입니다:

GridSearchCV를 사용하여 SVM 모델의 최적의 파라미터를 찾았습니다. 다음은 결과입니다:

- 최적의 파라미터:
  - `C`: 1
  - `gamma`: 0.1
  - `kernel`: sigmoid
- 교차 검증에서의 최적의 점수: 약 81.67%
- 테스트 세트에서의 성능: 약 63.64%

이 결과를 통해 SVM 모델의 최적의 파라미터를 찾았지만, 테스트 세트에서의 성능이 교차 검증 점수보다 낮게 나왔습니다. 이는 테스트 세트의 특성이나 노이즈, 데이터의 분포 등 다양한 요인으로 인해 발생할 수 있습니다.

또한 모델의 성능을 더 향상시키려면 다른 특성 선택 방법, 데이터 전처리 방법, 또는 다른 알고리즘을 고려해 볼 수 있습니다.

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# 파이프라인 생성
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(random_state=42))
])

# 파라미터 그리드 정의
param_grid = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'svm__gamma': ['scale', 'auto', 0.1, 1, 10]
}

# GridSearchCV 사용
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# 최적의 파라미터와 그 때의 점수
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# 테스트 세트에서의 성능 평가
best_estimator = grid_search.best_estimator_
test_accuracy = best_estimator.score(X_test, y_test)

best_params, best_score, test_accuracy


Fitting 5 folds for each of 80 candidates, totalling 400 fits


({'svm__C': 1, 'svm__gamma': 0.1, 'svm__kernel': 'sigmoid'},
 0.8166666666666667,
 0.6363636363636364)

In [7]:
# 전체 데이터 스케일링
X_scaled = std_scaler.fit_transform(X)

# SVM 모델 훈련
final_svm_model = SVC(kernel='linear', random_state=42)
final_svm_model.fit(X_scaled, y)

"모델이 성공적으로 훈련되었습니다."


'모델이 성공적으로 훈련되었습니다.'

### SVM 

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

# 1. Cross-Validation

X = merged_data.drop(columns=['date', 'VKOSPI_Label'])
y = merged_data['VKOSPI_Label']


# 5-fold SVM
cv_scores = cross_val_score(SVC(kernel='linear'), X, y, cv=5, scoring='accuracy')
cv_mean = cv_scores.mean()

cv_mean

0.6345454545454545

In [11]:
# 2. 하이퍼파라미터 튜닝

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

grid_search_cv = GridSearchCV(SVC(), param_grid, cv=5, n_jobs=-1)
grid_search_cv.fit(X, y)

best_params_cv = grid_search_cv.best_params_
best_score_cv = grid_search_cv.best_score_

best_params_cv, best_score_cv

({'C': 1, 'gamma': 'scale', 'kernel': 'sigmoid'}, 0.7254545454545454)

In [12]:
# 3. Feature Importance
best_svm_model = SVC(kernel='linear', C=best_params_cv['C'], gamma=best_params_cv['gamma'])
best_svm_model.fit(X, y)

feature_importances = best_svm_model.coef_

# 데이터프레임으로 정리
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances[0]
}).sort_values(by='Importance', ascending=False)

feature_importance_df


,Feature,Importance
10,kospi_p_range,1.088974
15,NAS_day_fluc_range,0.738331
6,cny_night_change,0.445919
21,sp_day_change,0.402114
16,niv_night_change,0.276642
27,vix_day_fluc_range,0.195732
5,cny_yesterday_change,0.092880
25,vix_close_change,0.045611
3,cd_p_change(%),-0.043847
14,NAS_day_change,-0.065680
